In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
from datetime import datetime

# Завантаження та підготовка даних
file_path = 'data/clear_currency_historical_data.csv'
data = pd.read_csv(file_path, delimiter=';')

# Перетворення дат та сортування
data['exchangedate'] = pd.to_datetime(data['exchangedate'], format='%d.%m.%Y')
data.sort_values('exchangedate', inplace=True)
data['timestamp'] = data['exchangedate'].astype(np.int64) // 10**9

# Закодування символу валюти
encoder = OneHotEncoder(sparse_output=False)
currency_encoded = encoder.fit_transform(data[['abbreviation']])

# Додавання нових ознак
currency_columns = [f'currency_{cat}' for cat in encoder.categories_[0]]
currency_df = pd.DataFrame(currency_encoded, columns=currency_columns)

# Об'єднання закодованої валюти з основними даними
data = pd.concat([data.reset_index(drop=True), currency_df.reset_index(drop=True)], axis=1)

# Формування вхідних даних
features = ['timestamp'] + currency_columns
X = data[features].values
y = data['rate_per_unit'].values

# Нормалізація вхідних даних
scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Розбиття на навчальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)

# Перетворення на тензори
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Визначення моделі LSTM
class LSTMCurrencyPredictor(nn.Module):
    def __init__(self, input_size):
        super(LSTMCurrencyPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=128, num_layers=2, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

# Ініціалізація моделі
input_size = X_train.shape[1]
model = LSTMCurrencyPredictor(input_size)

# Втрата та оптимізатор
criterion = nn.MSELoss()  # MSE для точного регулювання
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Навчання моделі
epochs = 200
batch_size = 32

# Перетворення X_train_tensor для LSTM (додавання виміру)
X_train_tensor = X_train_tensor.view(X_train_tensor.size(0), 1, -1)
X_test_tensor = X_test_tensor.view(X_test_tensor.size(0), 1, -1)

train_losses = []

# Використання DataLoader для батчевого навчання
train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

# Графік втрат
plt.plot(train_losses, label='Training Loss')
plt.legend()
plt.show()

# Функція для прогнозування
def predict_currency_rate(symbol, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    currency_code = encoder.transform([[symbol]]).flatten()
    predictions = []

    for date in date_range:
        timestamp = int(date.timestamp())
        features = [timestamp] + list(currency_code)
        features_scaled = scaler_x.transform([features])
        features_tensor = torch.FloatTensor(features_scaled).view(1, 1, -1)

        model.eval()
        with torch.no_grad():
            prediction_scaled = model(features_tensor)
            prediction = scaler_y.inverse_transform(prediction_scaled.numpy())
            predictions.append(prediction[0][0])

    return predictions

# Прогнозування на заданий період
symbol = 'USD'
start_date = '2024-12-01'
end_date = '2024-12-10'
predicted_rates = predict_currency_rate(symbol, start_date, end_date)

# Вивід прогнозу
print(f'Прогнозований курс для {symbol} з {start_date} по {end_date}:')
for date, rate in zip(pd.date_range(start=start_date, end=end_date), predicted_rates):
    print(f'{date.date()}: {rate:.2f}')



KeyboardInterrupt: 